In [ ]:
from pystac.client import Client
from odc.stac import load
import xarray as xr
from planetary_computer import sign_url

from odc.geo import BoundingBox

In [ ]:
# BBOX over Viti Levu, Fiji
bbox = BoundingBox(
    left=177.1,
    bottom=-18.4,
    right=178.8,
    top=-17.2,
    crs="EPSG:4326",
)

bbox.explore()

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/") 

In [ ]:
lulc_products = [
    "io-lulc-annual-v02",
    "esa-worldcover"
]

datasets = []

for product in lulc_products:
    items = catalog.search(
        collections=[product],
        intersects=bbox.polygon,
        datetime="2020-06",
    ).item_collection()

    print(f"Found {len(items)} items for product {product}")
    
    ds = load(
        items,
        intersects=bbox.polygon,
        crs="EPSG:6933",
        groupby="solar_day",
        resolution=100,
        chunks={"x": 2048, "y": 2048},
        patch_url=sign_url
    )

    print(f"Product {product} has variables: {ds.data_vars}")
    
    datasets.append(ds)

ds = xr.merge(datasets)
ds

In [ ]:
ds["io_lulc"] = ds["data"]
ds["esa_worldcover"] = ds["map"]

new_ds = ds.drop_vars(["data", "map", "input_quality.1", "input_quality.2", "input_quality.3"])

data = new_ds.squeeze().load()

In [ ]:
data.esa_worldcover.odc.explore()